# Follow up labs organized for Castor upload

### This lab layout follows the request of scientists and Castor staff  for upload in Castor

In [ ]:
#import libraries

In [ ]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

sys.path.insert(0, '../../')
from brickstudy import foundation

In [ ]:
### import example lab organization from scientst

In [ ]:
example = pd.read_csv('../../secret_data/Lab_vb_makeda.csv', sep=";")
example.head(4)

In [ ]:
example.columns

In [ ]:
example.shape

In [ ]:
# I reccomend turning K00 into K0, and NRCA to NRC0

### import first measurement moment after baseline labs from 2023 and 2024
These labs should be named with 1 and uploaded to Castor as repeating data

# SUPER IMPORTANT

This data has been mixed between multiple measurement moments. It needs filtering to get rid of baseline data. 

In [ ]:
#lab_data_baseline = pd.read_excel('../../secret_data/Brick_lab_20230914.xlsx')
lab_data = pd.read_csv('../../secret_data/BRICK_lab_data_update_18_03_2024.csv', sep= ';', encoding='latin1')

In [ ]:
lab_data.head(3)

In [ ]:
len(lab_data.studieID.unique())

In [ ]:
lab_data['studieID'] = lab_data['studieID'].astype(str)
lab_data['studieID'] = lab_data['studieID'].str.replace("-", "_")

In [ ]:
lab_data.head(3)

### We want  all the patients on thier own line, but not many lines per patient
We will try to accomplish this by splitting the dataset into each code-code-original i.e. each type of lab taken on the assumption we will almost always have one lab

In [ ]:
lab_data.code_code_original.unique()

In [ ]:
codes = lab_data.code_code_original.unique().astype(str)
codes

In [ ]:
lab_data[lab_data['code_code_original'].isna()]

Again we see all the nans are sodiums. We will transform them to NA

In [ ]:
lab_data['code_code_original'] = lab_data['code_code_original'].fillna('NA')

In [ ]:
codes = lab_data.code_code_original.unique().astype(str)
codes

In [ ]:
#lab_data[lab_data['code_code_original'] == 'nan']

In [ ]:
len(codes)

In [ ]:
## Notte there is a code BRICK...let's examine

In [ ]:
lab_data[lab_data.code_code_original =='BRICK'].head(5)

In [ ]:
len(example.columns)

Scientists not interested in ICT, LIP, HEM, ORD8, BAF9 or EX_LUMC therefore we will drop these values

In [ ]:
lab_data = lab_data.drop(lab_data[lab_data['code_code_original']== 'ICT'].index)
lab_data = lab_data.drop(lab_data[lab_data['code_code_original']== 'LIP'].index)
lab_data = lab_data.drop(lab_data[lab_data['code_code_original']== 'HEM'].index)
lab_data = lab_data.drop(lab_data[lab_data['code_code_original']== 'ORD8'].index)
lab_data = lab_data.drop(lab_data[lab_data['code_code_original']== 'BAF9'].index)
lab_data = lab_data.drop(lab_data[lab_data['code_code_original']== 'EX_LUMC'].index)

Also according to example there are about 50 other codes they do ont care for. We will drop these here too: 

In [ ]:
codes0 = [code + "0" for code in codes]
codes0_set = set(codes0)
expected_set = set(example.columns)
expected_set

In [ ]:
# here we will show columsn we were essentially asked to get rid of
droppable = codes0_set - expected_set
list_droppable = list(droppable)
list_droppable = [code[:-1] for code in list_droppable]
#list_droppable

In [ ]:
# here we can show what we will keep
list_keepable = expected_set
list_keepable =  [code[:-1] for code in list_keepable]
list_keepable

In [ ]:
list_keepable_amended = ['CRP',
                         'FERT',
                         'TROMBO',
                         'MCV',
                         'K',
                         'FOLZ',
                         'HB',
                         'HT',
                         'UREU',
                         'VITD',
                         'GGT',
                         'ALB_U',
                         'SCHWARTZ-Bedsite',
                         'ASAT',
                         'ERY',
                         'TBIL',
                         'DBIL',
                         'TE_U',
                         'TRAF',
                         'AFOS',
                         'NA',
                         'GRAT',
                         'LDH',
                         'FE',
                         'ALB_U_KR',
                         'B12',
                         'LEU',
                         'RETI',
                         'TSAT',
                         'KREA_U',
                         'ALAT',
                         'KREA',
                         'TE_U_KR',
                         'NRCA',
                         'RDW',]

In [ ]:
# lab_data_baseline = lab_data_baseline[~lab_data_baseline[lab_data_baseline['code_code_original'].isin(list_droppable)]]
# lab_data_baseline
lab_data = lab_data[lab_data['code_code_original'].isin(list_keepable_amended)]

In [ ]:
lab_data['code_code_original'].unique()

In [ ]:
new_codes = list(lab_data['code_code_original'].unique())

Scientist wants to get rid of doubles, i.e. #####brick 08, 12, 21 and 25. There are doubles because labs were not taken at precisely the same time. But we also do not want to upload duplicate data from the first measurement point to the second...therefore we can simply get rid of labs which are actually duplicates. 

In [ ]:
#example
lab_data[lab_data['studieID'] == 'BRICK_008']['effectiveDateTime'].unique()

All Brick 8 labs are duplicated from the first measurement moment. We can discard. 

In [ ]:
#12
lab_data[lab_data['studieID'] == 'BRICK_012']['effectiveDateTime'].unique()

All brick-12 measurements are duplicated from first baseline measurement moment, we can discard

In [ ]:
lab_data[lab_data['studieID'] == 'BRICK_021']['effectiveDateTime'].unique()

In [ ]:
lab_data[lab_data['studieID'] == 'BRICK_025']['effectiveDateTime'].unique()

All Brick 25 are duuplicates of first measurement moment. We can discard.

There are also additional duplicates in labs for 23 and 55

In [ ]:
lab_data[lab_data['studieID'] == 'BRICK_023']['effectiveDateTime'].unique()

These labs are effectively the same date, so we will make them so

In [ ]:
brick_23_wanted = lab_data[lab_data['studieID'] == 'BRICK_023']
# brick_23 seems to have a lot of botched labs they redid
brick_23_wanted = brick_23_wanted[brick_23_wanted['valueQuantity_value'].notna()]
brick_23_wanted

In [ ]:
lab_data[lab_data['studieID'] == 'BRICK_055']['effectiveDateTime'].unique()

These labs are effectively the same date, so we will make them so

In [ ]:
lab_data[lab_data['studieID'] == 'BRICK_055']

We must choose as the same labs were ralen more than once.

In [ ]:
brick_55_wanted = lab_data[lab_data['studieID'] == 'BRICK_055']
brick_55_wanted = brick_55_wanted[brick_55_wanted['effectiveDateTime'].isin(['2023-11-17 11:35:12.0000000', '2023-11-17 12:20:40.0000000'])]
# we also have different issue times, we will get rid of that
brick_55_wanted['issued'] = '2023-11-17 13:09:00.0000000'

In [ ]:
brick_55_wanted

In [ ]:
STOP #brick_55_wanted STOP

In [ ]:
lab_data[lab_data['studieID'] == 'BRICK_052']['effectiveDateTime'].unique()

In [ ]:
brick_52_wanted = lab_data[lab_data['studieID'] == 'BRICK_052']
# brick_52_wanted['effectiveDateTime'] = '2023-04-04 14:03:22.0000000'
# brick_52_wanted['issued'] = '2023-04-04 14:42:56.0000000'
for column in brick_52_wanted.columns:
    print(brick_52_wanted[column].unique())

In [ ]:
#now let's drop all of them, then append back on what we want
lab_data_no_ = lab_data.drop(lab_data[lab_data['studieID'] == 'BRICK_012'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_008'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_021'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_025'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_023'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_055'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_052'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_058'].index)

In [ ]:
lab_data_no_.studieID.unique()

In [ ]:
lab_data_no_.effectiveDateTime.unique()

In [ ]:
brick_58_wanted = lab_data_no_[lab_data_no_['studieID'] == 'BRICK_058']
brick_58_wanted['effectiveDateTime']= '2023-05-10 09:45:08.0000000'
brick_58_wanted['issued']= '2023-05-10 10:41:43.0000000'

In [ ]:
#lab_data_no_[lab_data_no_['effectiveDateTime'] == '2023-05-10 09:45:08.0000000']

# We see we still have lots of labs that are original, we will need to get rid off them by either a date cut-off or a filter

We also see some suspicoiusly close labs

In [ ]:
lab_data_no_

In [ ]:
lab_data = pd.concat([lab_data_no_, brick_52_wanted, brick_55_wanted, brick_23_wanted, brick_58_wanted])
lab_data.shape

In [ ]:
brick_55_wanted

In [ ]:
list_of_data_frames = []
for code in new_codes:
    prenames = ["code_code_original", code]
    frame_name = "_".join(prenames)
    #print(frame_name)
    frame_name = lab_data[lab_data['code_code_original'] == code]
    frame_name = frame_name.add_suffix(code)
    frame_name = frame_name.rename(columns= {frame_name.columns[0]:'studieID'})
    list_of_data_frames.append(frame_name)
    

In [ ]:
len(list_of_data_frames)

In [ ]:
len(new_codes)

Let's look at an example:

In [ ]:
list_of_data_frames[0].head(3) 

In [ ]:
list_of_data_frames[1].head(3) 

In [ ]:
list_of_data_frames[1].shape

So far so good, we have a frame with in the example above the DBIL. We then tried to do a big reduction; but it produced repeaters

In [ ]:
semi_final_df = reduce(lambda left,right: pd.merge(left,right, on=['studieID'], how='outer'), list_of_data_frames)

In [ ]:
semi_final_df.shape

In [ ]:
semi_final_df.head(5)

In [ ]:
for studieID in semi_final_df.studieID:
    print(studieID)

In [ ]:
# now we need to change the columns names to those expoected by scientist
for  f in semi_final_df.columns:
    print(f)

In [ ]:
list_keepable_amended_with_id = ["valueQuantity_value" + code for code in list_keepable_amended]
# [code + "0" for code in codes]
list_keepable_amended_with_id = list_keepable_amended_with_id + ['studieID']
list_keepable_amended_with_id

In [ ]:
final_amend = ['studieID',
               'valueQuantity_valueCRP',
                 'valueQuantity_valueFERT',
                 'valueQuantity_valueTROMBO',
                 'valueQuantity_valueMCV',
                 'valueQuantity_valueK',
                 'valueQuantity_valueFOLZ',
                 'valueQuantity_valueHB',
                 'valueQuantity_valueHT',
                 'valueQuantity_valueUREU',
                 'valueQuantity_valueVITD',
                 'valueQuantity_valueGGT',
                 'valueQuantity_valueALB_U',
                 'valueQuantity_valueSCHWARTZ-Bedsite',
                 'valueQuantity_valueASAT',
                 'valueQuantity_valueERY',
                 'valueQuantity_valueTBIL',
                 'valueQuantity_valueDBIL',
                 'valueQuantity_valueTE_U',
                 'valueQuantity_valueTRAF',
                 'valueQuantity_valueAFOS',
                 'valueQuantity_valueNA',
                 'valueQuantity_valueGRAT',
                 'valueQuantity_valueLDH',
                 'valueQuantity_valueFE',
                 'valueQuantity_valueALB_U_KR',
                 'valueQuantity_valueB12',
                 'valueQuantity_valueLEU',
                 'valueQuantity_valueRETI',
                 'valueQuantity_valueTSAT',
                 'valueQuantity_valueKREA_U',
                 'valueQuantity_valueALAT',
                 'valueQuantity_valueKREA',
                 'valueQuantity_valueTE_U_KR',
                 'valueQuantity_valueNRCA',
                 'valueQuantity_valueRDW', ]

In [ ]:
filtered = semi_final_df[final_amend]
filtered

In [ ]:
columns_dictionary = {'studieID':'Participant ID',
                      'valueQuantity_valueCRP':'CRP1',  
                      'valueQuantity_valueFERT':'FERT1',
                      'valueQuantity_valueTROMBO':'TROMBO1',
                      'valueQuantity_valueMCV':'MCV1',
                      'valueQuantity_valueK':'K1',
                      'valueQuantity_valueFOLZ':'FOLZ1',
                      'valueQuantity_valueHB':'HB1',  
                      'valueQuantity_valueHT':'HT1',
                      'valueQuantity_valueUREU':'UREU1',  
                      'valueQuantity_valueVITD':'VITD1',
                      'valueQuantity_valueGGT':'GGT1',  
                      'valueQuantity_valueALB_U':'ALB_U1',
                      'valueQuantity_valueSCHWARTZ-Bedsite':'SCHWARTZ-Bedsite1',  
                      'valueQuantity_valueASAT':'ASAT1',
                      'valueQuantity_valueERY':'ERY1',  
                      'valueQuantity_valueTBIL':'TBIL1',
                      'valueQuantity_valueDBIL':'DBIL1',  
                      'valueQuantity_valueTE_U':'TE_U1',
                      'valueQuantity_valueTRAF':'TRAF1',  
                      'valueQuantity_valueNA':'NA1',
                      'valueQuantity_valueAFOS':'AFOS1',
                      'valueQuantity_valueGRAT':'GRAT1',  
                      'valueQuantity_valueLDH':'LDH1',
                      'valueQuantity_valueFE':'FE1',  
                      'valueQuantity_valueALB_U_KR':'ALB_U_KR1',
                      'valueQuantity_valueB12':'B121',  
                      'valueQuantity_valueLEU':'LEU1',
                      'valueQuantity_valueRETI':'RETI1',  
                      'valueQuantity_valueTSAT':'TSAT1',
                      'valueQuantity_valueKREA_U':'KREA_U1',  
                      'valueQuantity_valueALAT':'ALAT1',
                      'valueQuantity_valueKREA':'KREA1',  
                      'valueQuantity_valueTE_U_KR':'TE_U_KR1',
                      'valueQuantity_valueNRCA':'NRCA1',  
                      'valueQuantity_valueRDW':'RDW1'}
filtered = filtered.rename(columns = columns_dictionary)
filtered

In [ ]:
filtered.columns

In [ ]:
filtered['Participant Status']= 'Enrolled'

In [ ]:
filtered['Site Abbreviation'] = 'ERA'

No idea where to get Participant Creation Date from ...it isn'tteh same as lab date, so let's leave it out.

In [ ]:
filtered.shape

In [ ]:
#filtered.to_csv('../../secret_data/castor_with_salt_second_measurement_moment_labs.csv')

In [ ]:
filtered

# We also need to filter away the baseline LABS!